In [1]:
# import os
# import sys
# import tarfile
# import time

# source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
# target = 'aclImdb_v1.tar.gz'

# def reporthook(count, block_size, total_size):
#     global start_time
#     if count == 0:
#         start_time = time.time()
#         return
#     duration = time.time() - start_time
#     progress_size = int(count * block_size)
#     speed = progress_size / (1024.**2 * duration)
#     percent = count * block_size * 100. / total_size
#     sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
#                     (percent, progress_size / (1024.**2), speed, duration))
#     sys.stdout.flush()

# if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    
#     if (sys.version_info < (3, 0)):
#         import urllib
#         urllib.urlretrieve(source, target, reporthook)
    
#     else:
#         import urllib.request
#         urllib.request.urlretrieve(source, target, reporthook)

100% | 80 MB | 1.54 MB/s | 52 sec elapsed

In [2]:
# if not os.path.isdir('aclImdb'):

#     with tarfile.open(target, 'r:gz') as tar:
#         tar.extractall()

In [4]:
# import pyprind
# import pandas as pd
# import os

# # change the `basepath` to the directory of the
# # unzipped movie dataset

# basepath = 'aclImdb'

# labels = {'pos': 1, 'neg': 0}
# pbar = pyprind.ProgBar(50000)
# df = pd.DataFrame()
# for s in ('test', 'train'):
#     for l in ('pos', 'neg'):
#         path = os.path.join(basepath, s, l)
#         for file in os.listdir(path):
#             with open(os.path.join(path, file), 
#                       'r', encoding='utf-8') as infile:
#                 txt = infile.read()
#             df = df.append([[txt, labels[l]]], 
#                            ignore_index=True)
#             pbar.update()
# df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:01


In [5]:
# import numpy as np

# np.random.seed(0)
# df = df.reindex(np.random.permutation(df.index))

In [6]:
# df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [9]:
import pandas as pd
import numpy as np
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array('')